In [66]:
#Libraries

import numpy as np
import pandas as pd


In [67]:
#Dataset

Dataset = pd.read_csv("sc_data.csv")

In [68]:
Dataset.head()

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632


In [69]:
Dataset.isnull().any(axis = 1)

0     False
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97    False
98    False
99    False
Length: 100, dtype: bool

In [70]:
Dataset.keys()

Index(['Product type', 'SKU', 'Price', 'Availability',
       'Number of products sold', 'Revenue generated', 'Customer demographics',
       'Stock levels', 'Lead times', 'Order quantities', 'Shipping times',
       'Shipping carriers', 'Shipping costs', 'Supplier name', 'Location',
       'Lead time', 'Production volumes', 'Manufacturing lead time',
       'Manufacturing costs', 'Inspection results', 'Defect rates',
       'Transportation modes', 'Routes', 'Costs'],
      dtype='object')

In [71]:
#Risk Score

sel_col = ['SKU', 'Stock levels', 'Lead times' ]
risk_data = Dataset[sel_col]

risk_data["Risk_score"] = risk_data["Lead times"] * (1-risk_data["Stock levels"])

F:\ANACONDA\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [72]:
risk_data = risk_data.sort_values(by = "Risk_score", ascending = False)
risk_data

,SKU,Stock levels,Lead times,Risk_score
68,SKU68,0,8,8
2,SKU2,1,10,0
34,SKU34,1,26,0
16,SKU16,2,5,-5
4,SKU4,5,3,-12
...,...,...,...,...
53,SKU53,96,25,-2375
5,SKU5,90,27,-2403
46,SKU46,92,29,-2639
77,SKU77,96,28,-2660


In [73]:
# EOQ

holding_cost = 0.2

def EOQ(Dataset):
    
    D = Dataset["Number of products sold"]
    
    S = Dataset["Costs"]
    
    H = Dataset["Number of products sold"] * holding_cost
    
    cal = (2* S * D)/H
    
    eoq = np.sqrt(cal)
    
    return eoq

In [76]:
Dataset["E_O_Q"] = EOQ(Dataset)



In [78]:
col = ['SKU','Order quantities', 'E_O_Q']

Dataset[col]



,SKU,Order quantities,E_O_Q
0,SKU0,96,43.330368
1,SKU1,37,70.927116
2,SKU2,88,37.672308
3,SKU3,59,50.475356
4,SKU4,56,96.095818
...,...,...,...
95,SKU95,26,88.253286
96,SKU96,32,43.444464
97,SKU97,4,73.493702
98,SKU98,27,93.925442


In [81]:
# Customer Segmentation

mean_rev = Dataset.groupby(['Customer demographics','Product type'])['Revenue generated'].mean().reset_index()

mean_rev

,Customer demographics,Product type,Revenue generated
0,Female,cosmetics,6954.854220
1,Female,haircare,6362.416642
2,Female,skincare,6095.470280
3,Male,cosmetics,5347.741374
4,Male,haircare,5059.992731
5,Male,skincare,7806.214493
6,Non-binary,cosmetics,5247.096332
7,Non-binary,haircare,5567.306726
8,Non-binary,skincare,4650.833889
9,Unknown,cosmetics,6335.182378


In [82]:
# Lead Time Optimization

avg_lt_by_mode = Dataset.groupby(['Transportation modes'])["Lead times"].mean().reset_index()
avg_lt_by_mode

,Transportation modes,Lead times
0,Air,18.269231
1,Rail,14.892857
2,Road,17.137931
3,Sea,12.176471


In [83]:
best_lt_by_mode = avg_lt_by_mode.loc[avg_lt_by_mode["Lead times"].idxmin()]
best_lt_by_mode

Transportation modes        Sea
Lead times              12.1765
Name: 3, dtype: object

In [84]:
avg_lt_by_route = Dataset.groupby(['Routes'])["Lead times"].mean().reset_index()
avg_lt_by_route

,Routes,Lead times
0,Route A,14.697674
1,Route B,17.216216
2,Route C,16.350000


In [85]:
best_lt_by_route= avg_lt_by_route.loc[avg_lt_by_route["Lead times"].idxmin()]
best_lt_by_route

Routes        Route A
Lead times    14.6977
Name: 0, dtype: object